In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib

/home/hawliet/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
train = pd.read_csv("../data/cars/train_data.csv", sep = "\t", encoding='utf-8')
test = pd.read_csv("../data/cars/test_data.csv", sep = "\t", encoding='utf-8')

In [3]:
y = train['price']
test_y = test['price']
X = train.drop(['price'], axis = 1)
test_X = test.drop(['price'], axis = 1)

#### Multilabel Encoding

In [8]:
def f(x, l):
    if type(x) is list:
        l.update(x)
class MultilabelEncoding(BaseEstimator, TransformerMixin):
    def __init__(self, col_name):
        self.col_name = col_name
        self.type_list = set()
    def fit(self, X_df, y=None):
        types_sr = X_df[self.col_name].str.replace('[/+]', ' ').str.split()
        types_sr.apply(f, args=(self.type_list,))
        self.type_list = list(self.type_list)
        return self
    def transform(self, X_df, y=None):
        transformed_df = X_df.copy()
        for i in range(len(self.type_list)):
            transformed_df[self.col_name + '_' + self.type_list[i]] = transformed_df[self.col_name].apply(lambda x: 1 if ((type(x) is str) and (self.type_list[i] in x)) else 0)
        transformed_df.drop(self.col_name, axis=1, inplace=True)
        return transformed_df

#### col add and drop

In [9]:
class ColAdderDropper(BaseEstimator, TransformerMixin):
    def __init__(self, num_top_brands = 10):
        # TODO
        self.num_top_brands = num_top_brands
        self.dropped_cols = ['url', 'name', 'model', 'weightTotal', 'fuelType', 'vehicleTransmission', 'modelDate']
    def fit(self, X_df, y=None):
        brand_col = X_df.brand.str.extract(r'([a-zA-z]+)', expand=False)
        self.brand_counts_ = brand_col.value_counts()
        brands = list(self.brand_counts_.index)
        self.top_brands_ = brands[:max(1, min(self.num_top_brands, len(brands)))]
        return self
    def transform(self, X_df, y=None):
        df = X_df.copy()
        brand_col = df.brand.str.extract("([a-zA-z]+)", expand=False)
        brand_col[~brand_col.isin(self.top_brands_)] = 'Others'
        df["brand"] = brand_col
        df.drop(self.dropped_cols, axis=1, inplace=True)
        return df

In [10]:
cat_multi = ['vEfuelType', 'driveWheelConfiguration']
cat_single = ['brand', 'eLabel', 'bodyType', 'vEengineType']

In [11]:
col_adderdropper = ColAdderDropper()

In [12]:
num_cols = []
for col in X.columns:
    if (not col in cat_single) and (not col in cat_multi) and (not col in col_adderdropper.dropped_cols):
        num_cols.append(col)

#### Hàm train và validate model

In [13]:
def train_and_val(full_pipeline, train_X, train_y, val_X, val_y):
    full_pipeline.fit(train_X, train_y);
    #print("n_brands:", len(col_adderdropper.top_brands_))
    pred_y = full_pipeline.predict(val_X)
    train_score = full_pipeline.score(train_X, train_y)
    val_score = full_pipeline.score(val_X, val_y)
    print(train_score, val_score)
    print("MSE =", np.round(np.mean((pred_y - val_y) ** 2), 3))
    print("MAE =", np.round(np.mean(np.abs(pred_y - val_y)), 3))
    return train_score, val_score

### các pipeline cần thiết cho pipeline cho preprocessing

In [14]:
imp_mean = SimpleImputer(strategy='mean') 
imp_mode = SimpleImputer(strategy='most_frequent')

encoding = OneHotEncoder(handle_unknown='ignore', sparse=False)
vEfuelType_encoder = MultilabelEncoding('vEfuelType')
driveWheelConfig_encoder = MultilabelEncoding('driveWheelConfiguration')

categorical_transformer = make_pipeline(imp_mode, encoding)

colTransform = ColumnTransformer(transformers=[('numerical', imp_mean, num_cols),\
                                               ('categorical', categorical_transformer, cat_single),
                                               ('vEfuelType', vEfuelType_encoder, ['vEfuelType']),
                                               ('driveWheelConfig', driveWheelConfig_encoder, ['driveWheelConfiguration'])])

colNormalize = StandardScaler()

#preprocessing = make_pipeline(col_adderdropper, colTransform, colNormalize)

### Full pipeline với MLPRegressor

In [15]:
best_n_brands = 80
mlpregressor = MLPRegressor(hidden_layer_sizes=(256, 512, 512, 256, ), solver='adam', learning_rate='adaptive'\
                            ,random_state=0, max_iter=500, early_stopping=True, verbose=1)
full_pipeline = make_pipeline(col_adderdropper, colTransform, colNormalize, mlpregressor)
full_pipeline.set_params(coladderdropper__num_top_brands=best_n_brands)
train_and_val(full_pipeline, X, y, test_X, test_y)

Iteration 1, loss = 98.19971331
Validation score: 0.694891
Iteration 2, loss = 45.39279857
Validation score: 0.890794
Iteration 3, loss = 35.74153083
Validation score: 0.919243
Iteration 4, loss = 33.95217416
Validation score: 0.935488
Iteration 5, loss = 32.16732118
Validation score: 0.940387
Iteration 6, loss = 27.45952570
Validation score: 0.943076
Iteration 7, loss = 27.50694067
Validation score: 0.933659
Iteration 8, loss = 23.51543182
Validation score: 0.945654
Iteration 9, loss = 23.85213672
Validation score: 0.946339
Iteration 10, loss = 21.13878786
Validation score: 0.925930
Iteration 11, loss = 28.39498006
Validation score: 0.958330
Iteration 12, loss = 19.60875726
Validation score: 0.963816
Iteration 13, loss = 18.06010007
Validation score: 0.964053
Iteration 14, loss = 17.97257090
Validation score: 0.956668
Iteration 15, loss = 19.09140831
Validation score: 0.906418
Iteration 16, loss = 18.29493092
Validation score: 0.966422
Iteration 17, loss = 16.36221133
Validation score

(0.9823796724995534, 0.9750759410460554)

#### save model

In [16]:
joblib.dump(full_pipeline, 'nn_80_final.pkl')

['nn_80_final.pkl']

### full pipeline với RandomForestRegressor

In [18]:
rfregressor = RandomForestRegressor(n_estimators=1024, random_state=0, verbose=1)
full_pipeline = make_pipeline(col_adderdropper, colTransform, colNormalize, rfregressor)
full_pipeline.set_params(coladderdropper__num_top_brands = 90)
train_and_val(full_pipeline, X, y, test_X, test_y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1024 out of 1024 | elapsed: 40.2min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1024 out of 1024 | elapsed:  5.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1024 out of 1024 | elapsed:  7.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1024 out of 1024 | elapsed:  7.3min finished


0.9923362344818045 0.9706492786139482
MSE = 27.165
MAE = 2.405


(0.9923362344818045, 0.9706492786139482)

In [19]:
# save model
# joblib.dump(full_pipeline, 'rf_1024.pkl')